In [1]:
import cv2
import keras
import pandas as pd
from keras.utils import Sequence
import numpy as np

In [2]:
y_test = pd.read_csv('../test_dataset_test/test.csv')

data_list = []
for i in y_test['ID_img']:
    #print(i)
    image = cv2.imread('../test_dataset_test/test/'+i)
    image = cv2.resize(image, (224, 224))
    data_list.append(image)

In [3]:
x_test_all = np.array(data_list).astype('float32') / 255.0

class DataGenerator(Sequence):
    def __init__(self, x_set, batch_size):
        self.x = x_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x
x_test = DataGenerator(x_test_all, 12)


model_1 = keras.models.load_model('1_EfficientNetb0_ep290_full.h5')
model_2 = keras.models.load_model('2_EfficientNetb0v2_ep301_full.h5')
model_3 = keras.models.load_model('3_EfficientNetb0v2_ep150_full.h5')
y_pred = model_1.predict(x_test)
y_pred += model_2.predict(x_test)
y_pred += model_3.predict(x_test)

179/179 [==============================] - 6s 26ms/step


In [4]:
print(np.argmax(y_pred[0]))

5


In [5]:
y_test = pd.read_csv('../test_dataset_test/test.csv')
for i in range(y_test.shape[0]):
    y_test['class'][i] = np.argmax(y_pred[i])
print(y_test)

y_test.to_csv('full.csv', index=False)

C:\Users\koshk\AppData\Local\Temp\ipykernel_500\2614757099.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test['class'][i] = np.argmax(y_pred[i])


        ID_img  class
0        0.jpg      5
1        1.jpg      5
2        2.jpg      1
3        3.jpg      1
4        4.jpg      6
...        ...    ...
2133  2133.jpg      4
2134  2134.jpg      1
2135  2135.jpg      1
2136  2136.jpg      3
2137  2137.jpg      0

[2138 rows x 2 columns]
